### Calling Python Script

In [1]:
# Python Scripts
%run -i "Scripts/script.py"

/Users/zakijefferson/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/IPython/core/magics/execution.py:816: DtypeWarning: Columns (11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  exit_ignore=exit_ignore)
Scripts/script.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_company_name_id.drop_duplicates(subset="name", inplace=True)
Scripts/script.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_company_name_id["name"] = a_company_name_id["name"].str.replace(i, "", case = False)
Scripts/script.py:78: SettingWithCopy

## Fuzzy Name Matching

### Example
- This is a small example of how fuzzywuzzy works

In [74]:
ratio = process.extractBests(a_company_name_id["name"][34], b_company_name_id["entity_name"][:23])

In [75]:
print(a_company_name_id["name"][34])
print("-"*40)
print(b_company_name_id["entity_name"][:23])

lans 
----------------------------------------
0                            lotsoff 
1                           laserspe 
2                      longwen group 
6                             lisi sa
8                            la gear 
9                     la quinta inns 
10              la quinta motor inns 
11                          la z boy 
13                           labarge 
14    laboratory  of america holdings
15                       laclede gas 
16                     laclede steel 
17                 lacrosse footwear 
18                la z boy casegoods 
21             lafarge north america 
22                    lake ariel ban 
23                    lake shore ban 
24                lakeland financial 
25               lakeland industries 
27                      lam research 
28                              lamr 
29                   lamonts apparel 
30                  lamson  sessions 
Name: entity_name, dtype: object


In [76]:
ratio

[('laserspe ', 68, 1),
 ('lakeland financial ', 68, 24),
 ('lakeland industries ', 68, 25),
 ('lamson  sessions ', 68, 30),
 ('lamr ', 50, 28)]

# ---------------------------------------------------------------------------------

## Making datasets smaller

In [116]:
# Removing rows
a_company_name_id_short = a_company_name_id.drop(a_company_name_id.index[600:])

In [117]:
# Only left 200 rows to play with
a_company_name_id_short.shape

(600, 2)

In [118]:
# Removing rows
b_company_name_id_short = b_company_name_id.drop(b_company_name_id.index[600:])

In [119]:
# Only left 200 rows to play with
b_company_name_id_short.shape

(600, 2)

## Creating a series

In [144]:
a_company_name_id_short.head()

,vendor_id,name
0,26303872,lhasa duns
1,143446948,long lewis ford
2,151786331,lechris
3,285649634,lisas kayaks
4,152312291,laura nunez villarte


In [145]:
b_company_name_id_short.head()

,b_entity_id,entity_name
0,000BFG-E,lotsoff
1,000P08-E,laserspe
2,000JF6-E,longwen group
6,000L15-E,lisi sa
8,000NY7-E,la gear


In [148]:
# Creating a series 
compare_names_a_b_series = pd.MultiIndex.from_product([a_company_name_id_short["name"],
                                               b_company_name_id_short["entity_name"]]).to_series()

In [121]:
compare_names_a_b_series.head()

name         entity_name   
lhasa  duns  lotsoff                 (lhasa  duns, lotsoff )
             laserspe               (lhasa  duns, laserspe )
             longwen group     (lhasa  duns, longwen group )
             lisi sa                  (lhasa  duns, lisi sa)
             la gear                 (lhasa  duns, la gear )
dtype: object

## Performing Fuzzy name matching

In [130]:
# Calculates fuzzy metrics and returns a series
def metrics(tup):
    return pd.Series([fuzz.partial_ratio(*tup),  # Uses the length matching to determine a score
                      fuzz.token_set_ratio(*tup),  # Calls ratio on three particular substring sets and returns the max
                      fuzz.token_sort_ratio(*tup)],  # Tokenizes the words and sorts them alphabetically to determine a score
                      ["ratio", "token_set", "token_sort"])

In [131]:
compare_names_a_b_metrics = compare_names_a_b_series.apply(metrics)

In [132]:
compare_names_a_b_metrics

ratio  token_set  token_sort
name        entity_name                                                   
lhasa  duns lotsoff                              38         24          24
            laserspe                             44         33          33
            longwen group                        18         26          26
            lisi sa                              43         47          47
            la gear                              50         35          35
...                                             ...        ...         ...
lisa bonura luna gold                            40         40          40
            lumina foundation for education      55         38          38
            logan capital management             45         23          23
            la jolla ve investors                45         38          38
            linux gold                           36         38          38

[360000 rows x 3 columns]

In [141]:
# Setting a threshold of 92% for the ratio and token
high_score_sort = compare_names_a_b_metrics[(compare_names_a_b_metrics["ratio"] >= 92) &
                                            (compare_names_a_b_metrics["token_set"] >= 92)]


In [142]:
high_score_sort

ratio  \
name                                               entity_name                           
liberty dialysis banksville                        liberty                         100   
liberty intl                                       liberty                         100   
                                                   liberty mint                     92   
lunar systems                                      lunar                           100   
liberty supply mpany dba                           liberty                         100   
lockheed martin  tactical systems                  lockheed martin                 100   
lockheed martin oration missiles and fire ntrol    lockheed martin                 100   
level                                              level one mmunications          100   
                                                   level jump financial group      100   
                                                   level  telem holdings           100   
                                                   level  systems                  100   
louisiana                                          louisiana land  exploration     100   
                                                   louisiana pacific               100   
                                                   louisiana casino cruises        100   
les installations lectriques pichette              les                             100   
lindsay nstruction                                 lindsay                         100   
ld                                                 ld holdings                     100   
les entreprises st louis ltee                      les                             100   
les solutions de rangement prisma                  les                             100   
les architectes goulet et lebel                    les                             100   
ledyard                                            ledyard financial group         100   
les expertises jacques desnoyers   ans             les                             100   
les services dentretiens sld                       les                             100   
les rnovations frangec                             les                             100   
ln holdings                                        lin holdings                     92   
les gestions na carrier                            les                             100   
les encans ritchie bros                            les                             100   
les ateliers kami                                  les                             100   
les entreprises gm laplante lte                    les                             100   
les cimentiers morissette                          les                             100   
les entreprises jacques dufour et fils             les                             100   
les services jfl  qubec                            les                             100   
lockheed martin po box                             lockheed martin                 100   
lauritzen  associates                              lauritzen                       100   
les aliments lbc                                   les                             100   
leidos innovations oration div                     leidos                          100   
lockheed martin global training and logistics      lockheed martin                 100   
lockheed martin oration  dba lockheed martin ae... lockheed martin                 100   
lockheed martin  dba lockheed martin aeronautics   lockheed martin                 100   
lifetime television                                lifetime                        100   
les puits atsiens deschnes                         les                             100   
lsi oration of america                             lsi                             100   
liberty tire auburndale recycling center           liberty                         100   
lockheed martin oration  dba lo

## Using Fuzzy matcher

In [165]:
import fuzzymatcher

In [170]:
# Defining which columns to match for the 2 dataframes
right_a = ["name"]
left_b = ["entity_name"]

matched_results = fuzzymatcher.fuzzy_left_join(df_left=b_company_name_id,
                                               df_right=a_company_name_id,
                                               left_on=left_b, right_on=right_a)

In [186]:
# Organizing dataframe
matched_results = matched_results[["best_match_score", "entity_name", "name", "vendor_id"]]

In [187]:
matched_results.sort_values(by=["best_match_score"], ascending=False, inplace=True)

In [188]:
matched_results.head(20)

,best_match_score,entity_name,name,vendor_id
606107,2.014880,lifeway christian resources of the southern ba...,lifeway christian resources of the southern ba...,49905136.0
833718,1.987137,lenczner slaght royce smith griffin llp,lenczner slaght royce smith griffin llp,310955696.0
1143395,1.973002,lutheran social services of wisnsin upper mic...,lutheran social services of wisnsin upper mic...,20015576.0
3595210,1.921255,lac vieux desert band of lake superior chippew...,lac vieux desert band of lake superior chippew...,136156015.0
1108097,1.851702,legal aid society of middle tennessee the cum...,legal aid society of middle tennessee and the ...,316986498.0
593121,1.846385,lac du flambeau band of lake superior chippewa...,lac du flambeau band of lake superior chippewa...,315431472.0
1095669,1.797841,la jolla institute for allergy immunology,la jolla institute for allergy immunology,23201929.0
3475171,1.781017,lower eastside girls club of new york,lower eastside girls club of new york,24210855.0
751138,1.778050,lucile salter packard childrens hospital at st...,lucile salter packard childrens hospital at st...,148580191.0
3702161,1.776409,lanka jathika sarvodaya shramadana sangamaya,lanka jathika sarvodaya shramadana sangamaya,23768003.0
